In [1]:
########### Python Form Recognizer Async Analyze #############
import json
import time
from requests import get, post

# Endpoint URL
endpoint = r"https://ocrdscale.cognitiveservices.azure.com/"
apim_key = "db8bf67aa2b3412d8f23d0484e227ec4"
model_id = "5d4f7bc5-c503-4a08-9070-7fc3fe8cecc0"
post_url = endpoint + "/formrecognizer/v2.0/custom/models/%s/analyze" % model_id
source = r"Test.pdf"
params = {
    "includeTextDetails": True
}

headers = {
    # Request headers
    'Content-Type': 'application/pdf',
    'Ocp-Apim-Subscription-Key': apim_key,
}
with open(source, "rb") as f:
    data_bytes = f.read()

try:
    resp = post(url = post_url, data = data_bytes, headers = headers, params = params)
    if resp.status_code != 202:
        print("POST analyze failed:\n%s" % json.dumps(resp.json()))
        quit()
    print("POST analyze succeeded:\n%s" % resp.headers)
    get_url = resp.headers["operation-location"]
except Exception as e:
    print("POST analyze failed:\n%s" % str(e))
    quit()

POST analyze succeeded:
{'Content-Length': '0', 'Operation-Location': 'https://ocrdscale.cognitiveservices.azure.com/formrecognizer/v2.0/custom/models/5d4f7bc5-c503-4a08-9070-7fc3fe8cecc0/analyzeresults/2d530ec6-32ea-404e-a150-edb85639c5d8', 'x-envoy-upstream-service-time': '83', 'apim-request-id': '3811ce2e-11d3-4c3f-912c-590f39b251dc', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'Date': 'Sun, 01 Nov 2020 12:07:10 GMT'}


In [2]:
n_tries = 15
n_try = 0
wait_sec = 5
max_wait_sec = 60
while n_try < n_tries:
    try:
        resp = get(url = get_url, headers = {"Ocp-Apim-Subscription-Key": apim_key})
        resp_json = resp.json()
        if resp.status_code != 200:
            print("GET analyze results failed:\n%s" % json.dumps(resp_json))
            quit()
        status = resp_json["status"]
        if status == "succeeded":
            print("Analysis succeeded:\n%s" % json.dumps(resp_json))
            quit()
        if status == "failed":
            print("Analysis failed:\n%s" % json.dumps(resp_json))
            quit()
        # Analysis still running. Wait and retry.
        time.sleep(wait_sec)
        n_try += 1
        wait_sec = min(2*wait_sec, max_wait_sec)     
    except Exception as e:
        msg = "GET analyze results failed:\n%s" % str(e)
        print(msg)
        quit()
print("Analyze operation did not complete within the allocated time.")

Analysis succeeded:
{"status": "succeeded", "createdDateTime": "2020-11-01T12:07:10Z", "lastUpdatedDateTime": "2020-11-01T12:07:17Z", "analyzeResult": {"version": "2.0.0", "readResults": [{"page": 1, "language": "en", "angle": 1.7579, "width": 8.2606, "height": 11.6772, "unit": "inch", "lines": [{"boundingBox": [3.0169, 1.8901, 4.3203, 1.9301, 4.317, 2.0701, 3.0135, 2.0301], "text": "United Arab Emirates", "words": [{"boundingBox": [3.0347, 1.8915, 3.4295, 1.9102, 3.4245, 2.0459, 3.0277, 2.0345], "text": "United", "confidence": 0.959}, {"boundingBox": [3.4577, 1.9113, 3.7493, 1.9215, 3.7459, 2.0554, 3.4529, 2.0468], "text": "Arab", "confidence": 0.958}, {"boundingBox": [3.7776, 1.9223, 4.3187, 1.9327, 4.318, 2.0728, 3.7743, 2.0563], "text": "Emirates", "confidence": 0.959}]}, {"boundingBox": [4.5236, 1.9968, 4.6703, 2.0134, 4.6536, 2.1634, 4.507, 2.1468], "text": "C", "words": [{"boundingBox": [4.5385, 1.998, 4.6379, 2.0093, 4.621, 2.1583, 4.5216, 2.1471], "text": "C", "confidence": 0.

In [3]:
resp_json

{'status': 'succeeded',
 'createdDateTime': '2020-11-01T12:07:10Z',
 'lastUpdatedDateTime': '2020-11-01T12:07:17Z',
 'analyzeResult': {'version': '2.0.0',
  'readResults': [{'page': 1,
    'language': 'en',
    'angle': 1.7579,
    'width': 8.2606,
    'height': 11.6772,
    'unit': 'inch',
    'lines': [{'boundingBox': [3.0169,
       1.8901,
       4.3203,
       1.9301,
       4.317,
       2.0701,
       3.0135,
       2.0301],
      'text': 'United Arab Emirates',
      'words': [{'boundingBox': [3.0347,
         1.8915,
         3.4295,
         1.9102,
         3.4245,
         2.0459,
         3.0277,
         2.0345],
        'text': 'United',
        'confidence': 0.959},
       {'boundingBox': [3.4577,
         1.9113,
         3.7493,
         1.9215,
         3.7459,
         2.0554,
         3.4529,
         2.0468],
        'text': 'Arab',
        'confidence': 0.958},
       {'boundingBox': [3.7776,
         1.9223,
         4.3187,
         1.9327,
         4.318,
    